In [ ]:
import json

import pandas as pd

from melitk.fda2 import inventory
from getpass import getpass

In [ ]:
token = getpass()
tiger_token = f"Bearer {token}"
inventory.init(token=tiger_token)

In [ ]:
artifact_names = [f'mocked_artifact_{i}' for i in range(1, 6)]
artifacts = [
    {},
    {123: {234: {345: {'alpha': -1, 'beta': 0}, 'default': {'alpha': 3, 'beta': 4}}}},
    {
        123: {234: {345: {'alpha': 2, 'beta': 3}, 'default': {'alpha': 3, 'beta': 4}}},
        123: {234: {345: {'alpha': 2, 'beta': 10}, 'default': {'alpha': 3, 'beta': 4}}}
    },
    {123: {234: {'default': {'alpha': 3, 'beta': 4}}}},
    {123: {234: {345: {'alpha': 3, 'beta': 4}}}}
]
artifacts = [json.dumps(artifact).encode('utf-8') for artifact in artifacts]
version = '0.0.1'

In [ ]:
for artifact_name, artifact_bytes in zip(artifact_names, artifacts):
    artifact = inventory.create_artifact(artifact_name=artifact_name, version=version, type_='fda.Bytes')
    artifact.save_from_bytes(artifact_bytes)

In [ ]:
artifact_input_name = 'test_ctr_beta_parameters'
artifact_input = next(inventory.filter(name=artifact_input_name))

In [ ]:
def next_patch(version: str) -> str:
    version_splitted = version.split('-')
    numbers = version_splitted[0].split('.')
    numbers[2] = str(int(numbers[2]) + 1)
    version_splitted[0] = '.'.join(numbers)
    version = '-'.join(version_splitted)
    return version

next_patch(artifact_input.version)

In [ ]:
creative_list = json.loads(artifact_input.load_to_bytes())

In [ ]:
creative_list

In [ ]:
artifact_output_name = 'prod_ctr_beta_parameters'
artifact_output = inventory.create_artifact(
    artifact_name=artifact_output_name,
    version='0.0.8-MLA-Production',
    type_='fda.Bytes',
    ttl=7
)

In [ ]:
artifact_output.save_from_bytes(json.dumps(creative_list).encode('utf-8'))

In [ ]:
artifact = next(inventory.filter(name=artifact_output_name))

In [ ]:
json.loads(artifact.load_to_bytes())